## Tool Calling with Agentic AI - AutoGen

### LLM Used - Granite3.0-8B

In this notebook we will learn how to use Tool Calling with Agentic AI in order to solve different problems.

Tool-calling agents expand the capabilities of an LLM by allowing it to interact with external systems. This approach empowers agents to dynamically solve problems by utilizing tools, accessing memory, and planning multi-step actions.

Tool calling agents enable:

1. Multi-Step Decision Making: The LLM can orchestrate a sequence of decisions to achieve complex objectives.
2. Tool Access: The LLM can select and use various tools as needed to interact with external systems and APIs.

This architecture allows for more dynamic and flexible behaviors, enabling agents to solve complex tasks by leveraging external resources efficiently.

In [1]:
!pip install autogen-agentchat~=0.2 autogen psutil --quiet
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mistral-common 1.5.2 requires pillow<11.0.0,>=10.3.0, but you have pillow 11.1.0 which is incompatible.
kfp 2.9.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.3 which is incompatible.
kfp-kubernetes 1.3.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.3 which is incompatible.
kfp-pipeline-spec 0.4.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.9.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.3 which is incompatible.
kfp 2.9.0 requires requests-toolbelt<1,>=0

In [2]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache

# INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
# MODEL_NAME = "granite30-8b"
# API_KEY= os.getenv('API_KEY')

INFERENCE_SERVER_URL = "http://localhost:8000"
MODEL_NAME = "ibm-granite/granite-3.0-8b-instruct"
API_KEY= "alanliuxiang"


In [3]:
# Configuration for the vLLM endpoint
local_llm_config = {
    "config_list": [
        {
            "model": MODEL_NAME,
            "api_key": API_KEY,
            "base_url": f"{INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

In [4]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]

def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [5]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=local_llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    # llm_config=local_llm_config,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

In [6]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

/opt/app-root/lib64/python3.11/site-packages/autogen/agentchat/conversable_agent.py:2611: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/opt/app-root/lib64/python3.11/site-packages/autogen/agentchat/conversable_agent.py:2531: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [ ]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 02-03 05:47:00] {573} WARNING - Model ibm-granite/granite-3.0-8b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Assistant (to User):

Let's break down the calculation step by step:

1. Subtract 32 from 232: 232 - 32 = 200
2. Divide 13312 by the result: 13312 / 200 = 66.56
3. Add 44232 to the result: 44232 + 66.56 = 44308.56
4. Multiply the result by 5: 44308.56 * 5 = 221542.8

So, the answer is 221542.8.

--------------------------------------------------------------------------------
User (to Assistant):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 02-03 05:47:01] {573} WARNING - Model ibm

In [ ]:
# from autogen import ConversableAgent

# agent = ConversableAgent(
#     "chatbot",
#     llm_config=local_llm_config,
#     code_execution_config=False,  # Turn off code execution, by default it is off.
#     function_map=None,  # No registered functions, by default it is None.
#     human_input_mode="NEVER",  # Never ask for human input.
# )

# reply = agent.generate_reply(messages=[{"content": "What is Agentic AI?", "role": "user"}])
# print(reply)

In [ ]:
# from autogen import UserProxyAgent, ConversableAgent


# chatbot = autogen.AssistantAgent(
#     name="chatbot",
#     system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
#     llm_config=local_llm_config,
# )

# # create a UserProxyAgent instance named "user_proxy"
# user_proxy = autogen.UserProxyAgent(
#     name="user_proxy",
#     is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=10,
#     code_execution_config={
#         "work_dir": "coding",
#         "use_docker": False,
#     },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
# )


# # define functions according to the function description


# # one way of registering functions is to use the register_for_llm and register_for_execution decorators
# @user_proxy.register_for_execution()
# @chatbot.register_for_llm(name="python", description="run cell in ipython and return the execution result.")
# def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
#     ipython = get_ipython()
#     result = ipython.run_cell(cell)
#     log = str(result.result)
#     if result.error_before_exec is not None:
#         log += f"\n{result.error_before_exec}"
#     if result.error_in_exec is not None:
#         log += f"\n{result.error_in_exec}"
#     return log


# # another way of registering functions is to use the register_function
# def exec_sh(script: Annotated[str, "Valid Python cell to execute."]) -> str:
#     return user_proxy.execute_code_blocks([("sh", script)])


# autogen.agentchat.register_function(
#     exec_python,
#     caller=chatbot,
#     executor=user_proxy,
#     name="sh",
#     description="run a shell script and return the execution result.",
# )

In [ ]:
# with Cache.disk() as cache:
#     # start the conversation
#     user_proxy.initiate_chat(
#         chatbot,
#         message="Can you give me a program to check the space in my system in python? Then execute it",
#         cache=None,
#         max_turns=4,
#     )


In [ ]:
# import os
# import psutil

# def check_memory():
#     memory_info = psutil.virtual_memory()
#     return {
#         'total': memory_info.total,
#         'available': memory_info.available,
#         'percent': memory_info.percent
#     }

# check_memory()